In [ ]:
import pandas as pd
import numpy as np

from bokeh.io import push_notebook, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.plotting import figure, show, output_file


import numpy as np
import colour
from scipy.special import betaincinv
from limix_plot import cycler_ as cycler
from collections import OrderedDict
from numpy import asarray as asa

In [ ]:
output_notebook()

In [ ]:
df = pd.read_pickle('chrom_null.pkl')
df.set_index(['gene', 'intron'], inplace=True)
df.sort_index(inplace=True)
df.head()

In [ ]:
def expected(n):
    lnpv = np.linspace(1/(n+1), n/(n+1), n, endpoint=True)
    return np.flipud(-np.log10(lnpv))

def xy(pv):
    
    return 

def rank_confidence_band(nranks):
    alpha = 0.01
    n = nranks
    k0 = np.arange(1, n+1)
    k1 = np.flipud(k0).copy()
    mean = k0 / (n + 1)
    return mean

def qqplot(p, method, color, df0, thr=1e-1, fill_alpha=0.2):
    
    pv = df0['%s-pval' % method].values[:]
    lpv = -np.log10(pv)
    lpv_sort = np.argsort(lpv)
    expected_lpv = expected(len(lpv))

    ok = pv[lpv_sort] <= thr
    
    gene = asa([i[0] for i in df0.index.values[lpv_sort]])
    intron = asa([i[1] for i in df0.index.values[lpv_sort]])
    
    source = ColumnDataSource(data=dict(
        xname=expected_lpv[ok],
        yname=lpv[lpv_sort][ok],
        gene=gene[ok],
        intron=intron[ok],
        snp_id=df0['snp_id'][lpv_sort][ok],
        pval=pv[lpv_sort][ok],
        pos=df0['pos'][lpv_sort][ok]
    ))
    
    p.circle('xname', 'yname', source=source, color=color,
             fill_alpha=fill_alpha, line_width=0, line_color=None,
            legend=method)
    
    mean = rank_confidence_band(len(lpv))
    me = [-np.log10(m) for m in mean]
    p.line([me[0], me[-1]], [me[0], me[-1]], color='black')
    p.legend.location = 'top_left'
    return p